In [ ]:
import re
import networkx as nx
from networkx.readwrite import json_graph
from fourlang.text_to_4lang import TextTo4lang
from fourlang.lexicon import Lexicon
from graphviz import Source
from scripts.parse_data import read
from tqdm import tqdm

In [ ]:
data_frame = read("de", graded=False)
data_frame

In [ ]:
text_to_4lang_en = TextTo4lang(lang="en")

In [ ]:
fourlang_votes = []
with open("en_result.txt", "r+") as f:
    for line in f:
        line = line.split()
        fourlang_votes.append(line[2])

In [ ]:
from nltk.corpus import wordnet as wn

In [ ]:
text_to_4lang_en = TextTo4lang(lang="en")
text_to_4lang_de = TextTo4lang(lang="de", port=5006)
text_to_4lang_it = TextTo4lang(lang="it", port=5007)

In [ ]:
graph_premise = text_to_4lang_en.process_text(data_frame.iloc[154].premise, method="expand", depth=3, blacklist=["in", "of", "on"]) #legyen-e expand
graph_hypothesis = text_to_4lang_en.process_text(data_frame.iloc[154].hypothesis, method="expand", depth=1, blacklist=["in", "of", "on"])
dot_graph_premise = graph_premise.to_dot()
Source(dot_graph_premise)

In [ ]:
print(asim_jac_nodes_hyp(graph_premise, data_frame.iloc[154].hypothesis))

In [ ]:
from networkx import algorithms
def asim_jac_nodes_hyp(graph_premise, hypothesis):
    if hypothesis in graph_premise.get_nodes():
        root_id = [node for node in graph_premise.G.nodes() if clear_node(node) == hypothesis][0]
        graph_premise_only_zero = copy.deepcopy(graph_premise)

        delete_list = []

        for edge in graph_premise_only_zero.G.adj.items():
            for output_node in edge[1].items():
                inner_delete_list = []
                for edge_type in output_node[1].items():
                    if edge_type[1]["color"]:
                        inner_delete_list.append(edge_type[0])
                for inner_del in inner_delete_list:
                    del output_node[1]._atlas[inner_del]
                if len(output_node[1]) < 1:
                    delete_list.append(output_node[0])
            for to_del in delete_list:
                if to_del in edge[1]._atlas:
                    print(to_del)
                    del edge[1]._atlas[to_del]
        if algorithms.has_path(graph_premise_only_zero.G, graph_premise.root, root_id):
            return 1.0
        else:
            return 0

In [ ]:
def asim_jac_nodes(graph_premise, graph_hypothesis):
    prem = set(graph_premise.get_nodes())
    hyp = set(graph_hypothesis.get_nodes())
    prem = set([i.lower() for i in prem])
    hyp = set([i.lower() for i in hyp])
    sim = hyp & prem
    if not sim or len(hyp) == 0:
        return 0
    else:
        return float(len(sim)) / len(hyp)

In [ ]:
import re
import copy
def clear_node(node):
    """
    Clears the node from the 4lang id parts
    :param node: the text to clear
    :return: the cleared text
    """
    return re.sub(r'_[0-9][0-9]*', '', node)

In [ ]:
text_to_4lang_de.parser_wrapper.load_from_dict()

In [ ]:
text_to_4lang_de.get_definition("Dach")

In [ ]:
graph_hypothesis = text_to_4lang_de.process_text("dachgeschoss", True, 2, [], True)
dot_graph_premise = graph_hypothesis.to_dot()
Source(dot_graph_premise)

In [ ]:
graph_hypothesis = text_to_4lang_de.process_text("dachgeschoss", True, 1, [], False)

In [ ]:
text_to_4lang_de.parser_wrapper.parse["Arbeitslosigkeit"]

In [ ]:
preds = []
preds_graded = []
for i in tqdm(range(len(data_frame))):
    index = i
    premise = data_frame.premise[index]
    hypothesis = data_frame.hypothesis[index]
    score = data_frame.score[index]
    graph_premise = text_to_4lang_en.process_text(premise, method="expand", depth=3, blacklist=["in", "of", "on"]) #legyen-e expand
    graph_hypothesis = text_to_4lang_en.process_text(hypothesis, method="expand", depth=1, blacklist=["in", "of", "on"])
    graph_premise.filter_graph("part")
    #graph_premise.filter_graph("on")
    #graph_premise.filter_graph("use")
    #pred = asim_jac_nodes_hyp(graph_premise, hypothesis)
    pred = asim_jac_nodes(graph_premise, graph_hypothesis)
    if pred == 1.0:
        preds.append(1)
    else:
        preds.append(0)

In [ ]:
for i, pred in enumerate(preds_0):
    if preds[i] != pred:
        print(str(i) + " " + str(pred) + " " + str(preds[i]))

In [ ]:
preds = []
preds_graded = []
for i in tqdm(range(len(data_frame))):
    index = i
    premise = data_frame.premise[index]
    hypothesis = data_frame.hypothesis[index]
    score = data_frame.score[index]
    graph_premise = text_to_4lang_de.process_text(premise, method="expand", depth=3, blacklist=["in", "auf"]) #legyen-e expand
    graph_hypothesis = text_to_4lang_de.process_text(hypothesis, method="expand", depth=1, )
    pred = asim_jac_nodes(graph_premise, graph_hypothesis)
    preds_graded.append(pred)
    if pred == 1.0:
        preds.append(1)
        if score == 0:
            print(premise + "\t" + hypothesis)
    else:
        preds.append(0)

In [ ]:
preds = []
preds_graded = []
for i in tqdm(range(len(data_frame))):
    index = i
    premise = data_frame.premise[index]
    hypothesis = data_frame.hypothesis[index]
    premise = re.sub("_", " ", premise)
    hypothesis = re.sub("_", " ", hypothesis)
    score = data_frame.score[index]
    graph_premise = text_to_4lang_it.process_text(premise, method="expand", depth=3, blacklist=["di", "su", "il"]) #legyen-e expand
    graph_hypothesis = text_to_4lang_it.process_text(hypothesis,method="expand", depth=1)
    pred = asim_jac_nodes(graph_premise, graph_hypothesis)
    preds_graded.append(pred)
    if pred == 1.0:
        preds.append(1)
    else:
        preds.append(0)

In [ ]:
def get_4lang_score(premise, hypothesis):
    graph_premise = text_to_4lang_en.process_text(premise, True, 3) #legyen-e expand
    graph_hypothesis = text_to_4lang_en.process_text(hypothesis, True, 1)
    graph_premise.filter_graph("part")
    graph_premise.filter_graph("on")
    graph_premise.filter_graph("use")
    pred = asim_jac_nodes(graph_premise, graph_hypothesis)
    return pred

In [ ]:
preds = []
for j in tqdm(range(len(data_frame))):
    index = j
    premise = data_frame.premise[index]
    hypothesis = data_frame.hypothesis[index]
    score = data_frame.score[index]
    
    hyp_syn_names_all = []
    hyper_premise_names_all = []
    
    premise_syns = wn.synsets(premise, lang="ita")
    hyp_syns = wn.synsets(hypothesis, lang="ita")
    """
    if len(premise_syns) > 0 and len(hyp_syns) > 0:
        en_premise = premise_syns[0].lemmas()[0].name()
        en_hyp = hyp_syns[0].lemmas()[0].name()
        fourlang_score = get_4lang_score(en_premise, en_hyp)
    else:
        fourlang_score = 0
    """
    
    
    for premise_syn in premise_syns:

        hyperpremise = set([i for i in premise_syn.closure(lambda s:s.hypernyms())])

        hyper_premise_lemmas = []
        for i in hyperpremise:
            lemmas = i.lemmas()
            for lemm in lemmas:
                hyper_premise_lemmas.append(lemm)

        hyper_premise_names = set([i.name() for i in hyper_premise_lemmas])
        hyper_premise_names_all += list(hyper_premise_names)
        
    for hyp_syn in hyp_syns:
        hyp_syn_lemmas = hyp_syn.lemmas()
        hyp_syn_names = set([i.name() for i in hyp_syn_lemmas])
        
        hyp_syn_names_all += list(hyp_syn_names)
    
    if (set(hyp_syn_names_all) & set(hyper_premise_names_all)) or fourlang_votes[index] == 1:
        preds.append(1)
    else:
        if score == 1:
            print(premise + "\t" + hypothesis)
        preds.append(0)

In [ ]:
with open("result_graded", "w+") as f:
    for i,pred in enumerate(preds):
        premise = data_frame.premise[i]
        hypothesis = data_frame.hypothesis[i]
        f.write(premise + " " + hypothesis + " " + str(pred) + "\n")

In [ ]:
older_preds = preds

In [ ]:
premise_syns = wn.synsets("servierer", lang='eng')
print(premise_syns)

In [ ]:
import numpy as np
from sklearn.metrics import precision_recall_fscore_support
precision_recall_fscore_support(data_frame.score.tolist(), preds)

In [ ]:
for i, pred in enumerate(preds):
    if pred == 1 and data_frame.iloc[i].score == 1:
        print(data_frame.iloc[i].premise + "\t" + data_frame.iloc[i].hypothesis)

In [ ]:
for i, premise in enumerate(data_frame.premise):
    defin = text_to_4lang_en.get_definition(premise)
    if defin and "use" in defin:
        print(premise + "\t" + data_frame.iloc[i].hypothesis)

In [ ]:
text_to_4lang_en.get_definition("beef")

In [ ]:
for i, pred in enumerate(preds):
    if preds[i] == 1:
        if pred != data_frame.score.tolist()[i]:
            print(pred)
            print(i)

In [ ]:
data_frame.iloc[405]

In [ ]:
preds.count(1)

In [ ]:
data_frame

In [ ]:
fourlang_votes = preds

In [ ]:
data_frame